In [ ]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.cross_validation import train_test_split
X = open('./input/train_X.txt','r',encoding = 'utf8').readlines()
Y = open('./input/train_Y.txt','r',encoding = 'utf8').readlines()
for i in range(len(X)):
    X[i] = X[i].strip('\n')
    Y[i] = Y[i].strip('\n')
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('./input/stopword.txt', 'r', encoding='utf-8').readlines()] 

with open('分词拿去做word2vect.txt','w',encoding = 'utf8') as file:
    for i in range(len(X)):
        des = X[i].replace(' ','')
        seg_list = jieba.lcut(des)
        word_list = [] 
        for seg in seg_list:
            if seg not in stopwords:
                word_list.append(seg)
        file.write((' ').join(word_list) +'\n')

from gensim.models import word2vec
sentences = word2vec.Text8Corpus('分词拿去做word2vect.txt')
model = word2vec.Word2Vec(sentences,size = 50,window = 4 ,min_count = 3)

## 求每一条商品描述的word2vec均值

X_word2vec = [] # 存放词向量的列表
for line in range(len(X)):
    sum = np.zeros((1,50))
    words = X[line].split()
    num = 1
    for word in words:
        try:
            temp = model[word]
        except:
            continue
        else:
            temp = temp.reshape((1,50))
            sum += temp
            num = num+1
    if num > 1:
        num -= 1
    sum /= num
    X_word2vec.append(list(sum[0]))

X_train,X_test,Y_train,Y_test = train_test_split(X_word2vec,Y,test_size = 0.2)

from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train,Y_train)
y_pre = clf.predict(X_test)
result = np.mean(y_pre==Y_test)

